# Plotting

In [1]:
progress = false

In the following, we’ll be using the
[Makie](https://docs.makie.org/v0.21/) ecosystem for plottig. There are
multiple major plotting ecosystems in Julia and it’s largely a matter of
personal preference which to use. However, some plotting tools and
packages only exist in one ecosystem, such as
[MixedModelsMakie.jl](https://palday.github.io/MixedModelsMakie.jl/v0.4/)
for several convenient plotting functions related to MixedModels.jl.

In [1]:
using CairoMakie # in Makie, you load a particular backend
using MixedModelsMakie

Most plotting functions come in two variants:

1.  “full-service” that generates a new plot for scratch
2.  mutating, that modifies an existing plot or plots into a
    user-provided `Figure` or `Axis` (a component of a figure).

Following the broader convention within Julia, the mutating variants
include a `!` in their name. It is quite common for the full service
variant to be a minimal wrapper around the mutating variant and for the
documentation of the full service variant to refer to the mutating
variant, stating that all arguments are forwarded. We’ll see a few
examples of this pattern in the following.

# A few models

We’ll use the same models from the introduction. They are repeated here
for convenience.

In [1]:
using MixedModels
insteval = MixedModels.dataset("insteval")
fm1 = fit(MixedModel,
          @formula(y ~ 1 + studage + lectage + service + (1|s) + (1|d) + (1|dept)),
          insteval; progress)

In [1]:
fm2 = fit(MixedModel,
          @formula(y ~ 1 + studage + lectage + service +
                      (1 | s) +
                      (1 + service | d) +
                      (1 + service | dept)),
          insteval; progress)

In [1]:
fm3 = fit(MixedModel,
          @formula(y ~ 1 + studage + lectage + service +
                      (1 | s) +
                      (1 + service | d) +
                      zerocorr(0 + studage | d) +
                      (1 + service | dept)),
          insteval; progress)

Additionally, we’ll use a model based on the `sleepstudy` dataset. The
data are from a study looking at the impact of sleep deprivation on
reaction time and has three variables:

-   `subj`: the participant
-   `reaction`: reaction time in milliseconds (mean across tasks)
-   `days`: days of sleep restriction

In [1]:
sleep = fit(MixedModel,
            @formula(reaction ~ 1 + days + (1 + days|subj)),
            MixedModels.dataset("sleepstudy"))

Minimizing 2    Time: 0:00:00 (55.18 ms/it)
  objective:  1790.1256369894543
Minimizing 57    Time: 0:00:00 (15.22 ms/it)

# Coefficient plots

## Fixed Effects

The function `coefplot` creates a plot of the coefficient estimates
along with associated confidence intervals.

In [1]:
coefplot(fm1)

Because the intercept is often on a different scale than categorical
predictors and is not of particular interest, `coefplot` also has an
option for not including it.

In [1]:
coefplot(fm1; show_intercept=false, color=:red)

We can use the mutating variant `coefplot!` to put the plots from all
models into a single axis for comparison purposes.

In [1]:
let f = Figure()
    ax = Axis(f[1, 1]; title="Comparison of estimates")
    coefplot!(ax, fm1; show_intercept=false, conf_level=0.68, label="fm1")
    coefplot!(ax, fm2; show_intercept=false, conf_level=0.68, label="fm2")
    coefplot!(ax, fm3; show_intercept=false, conf_level=0.68, label="fm3")
    axislegend(ax, "model"; merge=true, position=:rb) # _r_ight _b_ottom
    f
end

## BLUPs

The function `caterpillar` creates a similar plot of the BLUPs and their
associated prediction intervals. The name caterpillar comes from the
hairy appearance that occurs with large numbers of random effects. In
lme4, the comparable plot was called `dotplot`.

In [1]:
# select the grouping variable we want to plot
caterpillar(fm1, :dept)

When plotting the BLUPs associated with a grouping variable with a very
large number of levels, we can use `qqcaterpillar`, which combines a
caterpillar plot a QQ-plot like spacing on the y-axis in order to give a
better impression of the distribution of the random effects.

In [1]:
qqcaterpillar(fm1, :dept)

<img width=1000 height=800 style='object-fit: contain; height: auto;' src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAA+gAAAMgCAIAAAA/et9qAAAABmJLR0QA/wD/AP+gvaeTAAAgAElEQVR4nO3de3TcdZ3/8c8kk0unoUlb0kDa9AbYSosWCy54+QniBVaPgoJ60BVRBJU94IqX3UXdKu6uriIgq7Kgq6u4rmfXy+GmeAHWy4pUV9keQC4tpU0vtKGUNukkaTLf3x/BtpRSWpjmm/fk8fgr82UyebWOmWe//XZSyLIsAQAAY1td3gMAAICnJ9wBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAijm9YW/9KUvLVu2bPbs2XkNAACAvKxcufKoo4563/vet++fklu433777StWrGhtbc1rwFgzODhYqVSam5vzHkJN2bJly6RJk/JeQU0pl8vFYrGhoSHvIdSOoaGhwcHBUqmU9xBqytatW1taWgqFQt5DntKyZcv6+vpihPvcuXPnzp27ZMmSvAaMNb29vYODg1OmTMl7CLUjy7Lu7u6urq68h1BTenp6SqWSxqKKyuVyb29ve3t73kOoKd3d3Z2dnXV1Y/ey8GeQwWP3FwMAAOwg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3ACCI7dtTuZwqlbx3QD6KeQ8AAHg6WZZWrkybNqWUUrGYurrSlCl5b4LR5ow7ADDmrVv3eLWnlIaG0kMPpYGBXAdBDpxxB+DAuvLKK//t3/4t7xWMXZVKpVKpFIt7bZL+/t2vkGlsTHv/FMa3wcHBxsbGvd/nBz/4wYwZM0ZnT1V4xgNwYE2YMGHy5Ml5r2DsqlQqQ0NDT9NYfX1pePgJR5qb09NlGePZwMBAY2NjoVDYy33q6oJdeyLcATiwzjnnnHPOOSfvFYxd5XK5t7e3vb19b3d6+OHU3b3zZn19OvJI4c5edHd3d3Z2hkvzvRPuAMCY19GRtm9PGzemSiU1NaVZs1Q745BwBwAimDEjTZ+ehodd2s64VVN/fQAA1LJCQbUzngl3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAARQzHsAABDH5s3pscdSSqmtLbW25r0GxhfhDgDsm3Xr0tq1j3/c05NmzEgdHbkOgvFFuANQHX/4wx96enryXsEBU6mk5cufcGTp0nTYYalQeJYPPDAw0N/f3+r8/dgzbdq05z3veXmvYCfhDkB1fOxjH7vhhhvyXgFUzWmnnfa9730v7xXsJNwBqI5LLrnkwgsvzHsFB0ylklasSFm280ih4Ix7bZs2bVreE3gC4Q5AdSxatCjvCRxgDz+curt33pw5M7W3P/tHLZfLvb297dV4KKhtwh0A2DcdHalUSps3p5RSW1s66KC8B8H4ItwBgH120EF6HfLiBzABAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACqH64r1y5cmhoqOoPCwAA41mVw72/v/+YY45Zv359dR8WAADGuaqF+yOPPPKd73znNa95zSOPPFKtxwQAAEZULdzvu+++b3/72/X19dV6QAAAYIditR7o+OOP/8EPfrBy5co5c+ZU6zEBAIARVQv3p3X11VdfffXVO24eeuih8+bNczX8Dn19fdu3bx8cHMx7CLUjy7KNGzc2NDTkPYSasmnTpgkTJkyYMCHvIdSO/v7+vr6+4eHhvIdQUzZu3FhXV1dXN3bfQbG3t7elpWW/PmX0wv21r33t4sWLd9z893//9wkTJkyZMmXUBoxxjY2Ng4ODfkOooizLyuWyJxXVValUSqVSqVTKewi1o1wuNzQ0+GZFdY28Ao7lcH8GZ0BGL9w7Ozs7Ozt33Lz++utTSo2NjaM2YIwb+a3wG0IVZVnW0NDgSUV1Nf5J3kOoHcPDw55UVN3IK+BYDvdn8E9Dx+4vBgAA2EG4AwBAAFUO9+bm5pe97GVNTU3VfVgAABjnqnyN+yGHHHLbbbdV9zEBAACXygAAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQwej85FQAC6+tL3d1p27bU0JAOOSQdfHDeg4BxR7gDwNMZHEz335+Gh1NKaWAgPfRQqq9PkyfnPQsYX4Q7EMOXvvSlO++8M+8VpP7+/mKxWCyOs5ePcjn19T3hSENDam3NaU2tGRoaGhoaam5uzntIGIVC4aqrrsp7BTkYZ995gbB+9atf/fCHP8x7BSnLspRSoVDIe8joqlRSlu1+sL4+jyk1aJw+qZ6Furo64T4+CXcghm9961t5TyCllHp6ekqlUqlUynvI6OrtTffe+4QjhxySpk/PaU2tKZfLvb297e3teQ+Bsc67ygDA02lpSZ2daccp4UmT0qGH5joIGI+ccQeAfXDooWnq1FQup4aGNN7+wgEYG4Q7AOybxsbU2Jj3CGD8cqkMAAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADCHQAAAhDuAAAQgHAHAIAAhDsAAAQg3AEAIADhDgAAAQh3AAAIQLgDAEAAwh0AAAIQ7gAAEIBwBwCAAIQ7AAAEINwBACAA4Q4AAAEIdwAACEC4AwBAAMIdAAACEO4AABCAcAcAgACEOwAABCDcAQAgAOEOAAABCHcAAAhAuAMAQADFvAcAwJjR25vWrEn9/amxMR1ySJo8Oe9BADs54w4AKaWUBgbSAw+k3t40NJS2bUsrVqStW/PeBLCTM+7AWHfVVVetXLky7xU8rlwuF4vFhoaGvIccAL29qbf3CUeam1NbW05rxpGhoaHBwcFSqZT3kNw0NzcvWbIk7xUEINyBse7666//9a9/nfcKHpdlWUqpUCjkPeQAqFRSlj3hSKGQ6vzV9A

In [1]:
qqcaterpillar(fm1, :d)

When a grouping variable is associated with multiple experimental
variables, then each receives its own panel in the caterpillar plot.

In [1]:
caterpillar(fm2, :dept)

By default, the levels of the grouping variable are sorted by their
value for the first column. However, we can select which variables are
displayed and which column is used for sorting.

In [1]:
caterpillar(fm2, :dept; cols=["(Intercept)", "service: Y"], orderby=2)

Because caterpillar plots can contain multiple axes, they cannot be
plotted directly into an axis, but they can be plotted into a
`GridLayout` (i.e. a sublayout) within a `Figure`.

In [1]:
let f = Figure(; title="Random effects")
    caterpillar!(f[1, 1], fm2, :dept)
    Label(f[0, 1], "dept"; tellwidth=false)
    qqcaterpillar!(f[1, 2], fm2, :d)
    Label(f[0, 2], "d(ozent)"; tellwidth=false)
    f
end

# General regression diagnostics

## QQ Plots

In [1]:
qqnorm(fm1)

In [1]:
qqplot(Normal(0, fm1.σ), fm1)

<img width=672 height=480 style='object-fit: contain; height: auto;' src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAqAAAAHgCAIAAAD17khjAAAABmJLR0QA/wD/AP+gvaeTAAAgAElEQVR4nO3deXxU9bn48efMTDJZJhsQIlvCviphUQEVEYyAImq16O11QasXl59V61i9LCUg2oqUW7derFRvr6Vqq17REdRaBSouCCgGQVAEDJiQhCXLZJ/l90c0xsDMnISzzJz5vP9yZk6mT18RPp7le44SDAYFAABYi83sAQAAgPYIPAAAFkTgAQCwIAIPAIAFEXgAACyIwAMAYEEEHgAACyLwAABYEIEHAMCCCDwAABZE4AEAsCACDwCABRF4AAAsiMADAGBBDrP+h//7v/97+/btffv2NWsAAABixf79+0877bTbbrtN/Y+YFviPPvpo7969GRkZZg2ADqmpqXG5XIqimD0IdFddXZ2enm72FNBdMBj0er1paWlmDwJVtm/fXltbGxuB79+/f//+/RctWmTWAOiQgwcP9uzZ02bjnI71FRcX5+bmmj0FdBcIBEpKSnr37m32IFClE7nk72sAACyIwAMAYEEEHgAACyLwAABYEIEHAMCCCDwAABZE4AEAsCACDwCABRF4AAAsiMADAGBBBB4AAAsi8AAAWBCBBwDAggg8AAAWZNrjYgEAsBhfIPjExn0vFZX6/MFLTs25e9KAJIdpO9IEHgAAbdz3+s7/2rC35Z83FR/bVeZ99t9HmzUMh+gBANBAbZP/8Y37277zl60Hv61qMGkcAg8AgBaqGpqb/YF2b5Z7G00ZRgg8AACaOCXN2Ssjqe076UmOIdkus+Yh8AAAaMCmKM9cNSo10d7y0umwPTUrP+X7l8bjIjsAALQxdUj27v+csvaLMn9Apg7J7t81xcRhCDwAAJrplZH0H+PzzJ5ChEP0AABYEoEHAMCCOEQPAEBIH+4/9petB+ua/VMGdrtmbC+bopg9kVoEHgAAEZEDlfX7j9YPyk49Jc3Z8s7LRaWznt0SDIqI/O/mAxu+PvL0VflmjtgRBB4AEO+CQbnlpaKVm74JBsVhU+6bMvCBC4eKyD2enS11b/HMx8V3T+o/4pQ00wbtCM7BAwDi3cpN3zz10TctLfcFgg/+8yvPjrJAMPjNsbp2W+472v6dqMUePAAgHpVUN/xly8GK2qYJeVlv7ipv9+kbu8pnjsgZ0DV1z+Hatu8P7JZq4IwnhcADAOLO54dqznpsY02jr+VlXlb7O9IkJ9hE5PeXjrjsfzb7A98dpv9/Z/cd2t20W892FIEHAFjc7nLv859+623yi8g7X1WUVDUGgsHWuovIN8fqFEVaT7cn2G1Xj+ktIhcPz9n6y3P/+snB6gbfBYOzLz+thxnjdxKBBwBYVkl1w9+3ldz7+hfHP+etnatH996w98iByvpB3VIfnjl8TO+Mlvfze6bn9xyu/6TaI/AAAAsKBuXO1Z//4f39gbbXwYd25aief7l6dKMv4HRY5PJzAg8AsKD/3XLg8Y37VG58Tr8uFw3rLiKWqbsQeACAxTT7A799Z8+ydXvCb6YoMmNYjqLI2X27/OKcfnZbzNyiTiUCDwCwAl8guOHrI58fqv7btpIP9x8Lv7FNUeYXDLp/+hBDRjMHgQcAxLDSqoZnNh/4380H9h6p84c+3W5TZNKAblfm9xjS3VXubRrdK2NwdsysaO8cAg8AiD2BYPDR9/Y9/M6eQ97GiBvnpDkf/8mps/J7GjBY9CDwAIAYU9PgO/PR93aVe9VsnJpoL7pnUneXU++pog2BBwDEkkM1jaOWbyiribzjLiLJCfaVV+bHYd2FwAMAYsvNLxaprPvZ/bq8PPv0nLR4rLsQeABADHlv71HPzrIwGzhsSnaqc2C3lCvye9x2Vt8Eu3XWtXcUgQcAxIBdFd7bXipa//WRE14pn5Jonzyg6y8nDZgysJtitQXtnUTgAQBRZ/+xupWbit//+ujXR2uP1TXXN7c+0a09p8P2wjVjLzvtFGMHjAEEHgAQRUqqGsY/tvFAZb3K7T03jbtgUDddR4pRBB4AEC3+uvXgdc9/Gmpn/XhTh2RT91AIPADAfIFg8OKVH7+xu1z9j/RIS/q/68/Qb6RYR+ABAGYqOlQ17clNh9StfGs1NMf1+o1npibadZrKAgg8AMAEr+44dP3z2yrrmzv0UzZFGZeb9fjlI8b2ztRpMMsg8AAAQ71cVPJvqz7x+dudaQ+KKMe/o4gkOmyZSQkje6RdNabn1WP6JFnoke26IvAAAIPc/PfPVm46EJQTXkTXfvW6TVHW3DRu+tDuBgxmSQQeAKCvJr8//3f/UvlsmBZpTsf+Bed3SUnUbyrLI/AAAL14m3z9lrxzuK5J/Y8oilxxWs/nrhkdz3eZ1QSBBwBoLxiUkb/b8PmhavU/4nTYnrz8tOvH5eo3VVwh8AAALa3aeuCG5z/znfCW8SeSYLPNOSt3+cUjnAnssmuJwAMAtHG0tjm78K2A6rQ7Hba3b54wsX8XXaeKWwQeAKCBKX/4YN3eIyo3zkxJKPn1BcncpkZPBB4AcLKc965t8vsjbRUUUXplJX1617nZLqcRY8U3Ag8A6KTdh2pOXb7Bp+7hMGN7ZW65+1y9R0IrAg8A6Ixei/5Rou4G8hnJCUfvn27jEjpjEXgAQMf88tUdj/xrb9hNvrvvbLYr8cM7Jg7ommLMYGiLwAMAOiBrwVuV9RFvXKOc2y9rw+3nGDEQQiDwAABVKuubuyx4M+L5dkWR/fMuyO2SZMRMCI3AAwAie31n+cynN0XczGm31T00g9Pt0YDAAwDCqa5vzvr1mxGvlHfalU/dk4blpBkyFCIj8ACAkI54G7sV/iP05989sr100dScNJa2RxcCDwA4gep6X+aCN0M8u72VctfE/r+/bIRBM6EjCDwA4EcamwOp89b4AxE2U0SqHrgoLZnbzUYpAg8A+IH9ntfVPC3G6ZCGpTMNmAedxpWOAAARkbmv7VTcHjV1H5SdSt2jH3vwAAA5pfDNMm9zxM0UkcBy0h4b2IMHgLj2xq5DNrdHTd0T7TbqHkPYgweA+OW8b22Tr+Uxr9/dPf6EbIriW3axEvJzRCMCDwDxqLSqIff3n7Y5337ieisim+4464y8rgaNBe0QeACIO4m/er050q3pFJHDi6d3cSUYMxI0R+ABII4Eg2K/xxPxQvme6UnfFl5gxEDQDYEHgHiRfN/rDb7Iq+AuPbX76hvGGTAPdEXgASAu2NyRd9xtIn6uk7cKAg8AFpe24A1vvS/iZs9dM/pno3sbMA+MQeABwLIamgIpc9dE3HHn9jWWROABwJrsbk+k58WIiFw58pS/zT5D92lgOAIPAFYzevm6bSXeiJspIr5lM23c0dSiCDwAWIqai+lE5I6z+//qzAzqbmEEHgAswvkrT5OKg/KKiP93M4PBQElJif5DwTQEHgBiXt/Ct4u9DWp23C8a1HXNLWeJiIqnwiK2EXgAiGF+vyTcq+qYPGfc4w2BB4BYpfI6eUVk620TRw/I1H0gRBMCDwCxJ33emzWNkZ/gLiIpDlvt0hl6z4MoROABIMao33E/tKige1qy7gMhKhF4AIgZSfeuafSribuckppUej+Pg4trBB4AYsCbm8sueuFjNRfT8cAYtCDwABDt1B+T9y6ZmZKi+zyICSyYAIDo9Z8vbbGpqLsi8vBFgwPLqTt+wB48AEQplTedTRZb3XKuk0d7BB4Aok6i26NmDZwiUnT7lFP7peo+EGIQgQeAKNKz0HPIK2p23JMcUr+Ui+kQkpaBX79+/erVq2tqasaPH3/DDTc4HPzXAwColf/Qu9sralXedDbAdfKIRLOL7FatWnXhhRfa7faBAwcuXbp02rRpgYCqxZoAAJvbU6Su7reckUvdoYZmO9kPPvjgwoUL586dKyKzZ8/Oy8v

## Multiple diagnostics

In [1]:
function diagnostic_plot!(f, model)
    ax = Axis(f[1, 1]; xlabel="fitted", ylabel="observed",
              title="Observed vs fitted", aspect=AxisAspect(1))
    scatter!(ax, fitted(model), response(model); alpha=0.5)
    ablines!(ax, 0, 1; linestyle=:dash)

    ax = Axis(f[1, 2]; xlabel="fitted", ylabel="residual",
             title="Residuals vs fitted")
    scatter!(ax, fitted(model), residuals(model); alpha=0.5)
    hlines!(ax, 0; linestyle=:dash)

    ax = Axis(f[2, 1]; xlabel="theoretical quantiles", ylabel="residuals",
             title="Normal QQ", aspect=AxisAspect(1))
    qqnorm!(ax, model)

    ax = Axis(f[2, 2]; xlabel="Residual value", ylabel="density",
             title="Residuals")
    density!(ax, residuals(model))

    Label(f[0, :], "Regression diagnostics";
          tellwidth=false, fontsize=24)

    colsize!(f.layout, 1, Auto(0.5))

    return f
end

diagnostic_plot!(Figure(), sleep)

# Shrinkage

One of the most powerful aspects of mixed effects models is *shrinkage*,
i.e. the regularization entailed in the random effects. `shrinkageplot`
displays a comparison of the values obtained for each level of the
grouping variable based on a mixed models fit and a fit comparable to
traditional ordinary least squares.

In [1]:
shrinkageplot(fm2, :dept)

While it may look like some values go further away from the grand mean,
plotting the associated correlation ellipses (computed from the
correlation estimates in the mixed model) shows that these values are on
a level-curve closer to the grand mean than where they started.

In [1]:
shrinkageplot(fm2, :dept, ellipse=true, ellipse_scale=10, n_ellipse=10)

When there are multiple predictors associated with a given grouping
variable, then `shrinkageplot` displays all pairwise combinations.

In [1]:
shrinkageplot(fm3, :d, ellipse=true)

In the case of overparameterized random effects, one or more dimensions
will be degenerate and the corresponding panel will show a line or a
point mass instead of a cloud of points.

Similar to `caterpillar`, `shrinkageplot` uses multiple axes internal
and so must be plotted directly into a `Figure`.

In [1]:
let f = Figure()
    Label(f[0, 1], "d(ozent)"; tellwidth=false)
    shrinkageplot!(f[1, 1], fm2, :d)
    Label(f[0, 2], "dept"; tellwidth=false)
    shrinkageplot!(f[1, 2], fm2, :dept)
    f
end

In [1]:
let f = Figure()
    Label(f[0, 1], "d(ozent)"; tellwidth=false)
    shrinkageplot!(f[1, 1], fm3, :d)
    Label(f[0, 2], "dept"; tellwidth=false)
    shrinkageplot!(f[1, 2], fm3, :dept)
    f
end